In [3]:
#tensorboard
"""
1. conda info --envs
2. conda activate base
3. tensorboard --logdir=./~mylogs
4. open http://localhost:6006/
"""

'\n1. conda info --envs\n2. conda activate base\n3. tensorboard --logdir=./~mylogs\n4. open http://localhost:6006/\n'

In [5]:
import tensorflow as tf
import numpy as np
import datetime

x = np.array([[0,0],[0, 1],[1, 0],[1, 1]], dtype = np.float32)
y = np.array([[0], [1], [1], [0]], dtype = np.float32)

w1 = tf.Variable(tf.random.normal([2, 2]))
b1 = tf.Variable(tf.random.normal([2]))

w2 = tf.Variable(tf.random.normal([2, 2]))
b2 = tf.Variable(tf.random.normal([2]))

w3 = tf.Variable(tf.random.normal([2, 1]))
b3 = tf.Variable(tf.random.normal([1]))

def sigmoid(x):
    return 1 / (1+tf.exp(-x))
@tf.function
def perceptron(x):
    hidden = sigmoid(tf.matmul(x, w1) + b1)
    hidden2 = sigmoid(tf.matmul(hidden, w2) + b2)
    return sigmoid(tf.matmul(hidden2, w3) + b3)
@tf.function
def loss_fn(y_true, y_pred):
    return tf.reduce_mean(-(y_true * tf.math.log(y_pred) + (1 - y_true)*tf.math.log(1 -y_pred)))

optimizer = tf.keras.optimizers.SGD(learning_rate = 0.5)

def binary_output(x):
    return int(x.numpy()[0][0] > 0.5)

def accuracy(y_true, y_pred):
    correct_prediction = tf.equal(tf.cast(y_pred > 0.5, tf.float32), y_true)
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Log save path - tensorboard
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = f'logs/mylogs/{stamp}'
writer = tf.summary.create_file_writer(logdir)

#Graph tracking and log saving - tensorboard
tf.summary.trace_off()
tf.summary.trace_on(graph=True)
pred_initial = perceptron(x)
_ = loss_fn(y, pred_initial)
with writer.as_default():
    tf.summary.trace_export(name="graph_trace", step=0)
print(f"log saved -> {logdir}")

for epoch in range(5000):
    with tf.GradientTape() as tape:
        pred = perceptron(x)
        loss = loss_fn(y, pred)
    gradients = tape.gradient(loss, [w1, b1, w2,b2, w3, b3])
    optimizer.apply_gradients(zip(gradients, [w1, b1, w2,b2, w3, b3]))

    acc_val = accuracy(y, pred)

    with writer.as_default():
        tf.summary.scalar("Loss", loss, step=epoch)
        tf.summary.scalar("Accuracy", acc_val, step=epoch)
        
    if (epoch+1) %500 == 0:
        w1_val = np.squeeze(w1.numpy())
        b1_val = np.squeeze(b1.numpy())
        w2_val = np.squeeze(w2.numpy())
        b2_val = np.squeeze(b2.numpy())
        w3_val = np.squeeze(w3.numpy())
        b3_val = np.squeeze(b3.numpy())
        print(f"step = {epoch+1}, cost = {loss.numpy():.9f}")
        print(f"w = {w1_val}, b = {b1_val}")
        print(f"w = {w2_val}, b = {b2_val}")
        print(f"w = {w3_val}, b = {b3_val}")
for input_data in x:
    prediction = perceptron(np.array([input_data], dtype=np.float32))
    print(f"Input: {input_data.tolist()}, Prediction(%): {prediction.numpy()[0][0]:.4f}, Prediction(Binary): {binary_output(prediction)}")

#Final Acc
predictions = perceptron(x)
acc = accuracy(y, predictions)
print(f"Acc: {acc.numpy():.4f}")

#log saved
print(f"log saving path - TensorBoard: {logdir}")

2025-05-07 00:04:14.464412: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


log saved -> logs/mylogs/20250507-000420
step = 500, cost = 0.680882215
w = [[-0.78266335 -1.4110566 ]
 [-0.52949774  1.1748805 ]], b = [-0.19011432  1.5371673 ]
w = [[-0.16363601  0.26892635]
 [-1.4252124   0.77568686]], b = [0.10529738 0.5206397 ]
w = [ 1.637362  -1.3991512], b = 0.6424564123153687
step = 1000, cost = 0.510709465
w = [[-0.96508104 -3.9975019 ]
 [-1.033962    4.4686246 ]], b = [-0.36387217  2.5336547 ]
w = [[-0.33569017  0.3867107 ]
 [-4.123049    2.1699014 ]], b = [ 1.4336555  -0.32731846]
w = [ 4.3774834 -2.306686 ], b = 1.0803422927856445
step = 1500, cost = 0.467125982
w = [[-0.8383455 -4.8056555]
 [-2.3190737  5.632705 ]], b = [-0.11207987  3.4285545 ]
w = [[-1.0604297  1.3521061]
 [-4.7671714  2.0452023]], b = [ 2.2080424 -1.0479528]
w = [ 5.483683  -2.9054606], b = 1.2337948083877563
step = 2000, cost = 0.126198471
w = [[ 2.217181 -5.413704]
 [-3.137639  6.840512]], b = [1.5515332 3.4859676]
w = [[-1.9492263  3.1854181]
 [-4.17587    1.6521438]], b = [ 4.118602